# Data Masters Case: Modelo de Classificação

Felipe Viacava -- São Paulo, ago/2023

O presente documento consiste no treinamento e avaliação de modelos de classificação como parte da solução do Case "Data Masters - Cientista de Dados" do Santander Brasil. O objetivo é treinar um modelo de classificação de clientes possivelmente insatisfeitos com o banco para selecionar o público alvo de uma campanha de retenção.

Além das premissas estabelecidas na etapa de exploração dos dados, assumimos que:
- Buscamos um modelo de alto poder preditivo e não um modelo para inferência;
- Para cada cliente marcado como insatisfeito (_predicted positive_) pelo modelo, temos um custo de R$10,00;
- Destes, para os verdadeiramente insatisfeitos (_true positives_), temos um retorno de R$100,00;
- O melhor modelo é aquele que maximiza o retorno sobre a campanha de retenção, ou seja, a soma de:
    - _False Positives_, que geram custo R$10,00 por observação;
    - _True Positives_, que geram lucro de R$90,00 por observação;
    - _False Negatives_, que não geram lucro nem custo por observação;
    - _True Negatives_, que não geram lucro nem custo por observação.

As premissas acima surgem da interpretação do enunciado do problema. Outras premissas que nortearam o desenvolvimento dos modelos são:
- O código deve ser limpo, conciso e reprodutível;
- A performance dos modelos deve ser comparada sobre um conjunto de dados nunca antes visto, prevenindo _data leakage_. Este conjunto foi separado antes da análise exploratória, simulando o que seria o acompanhamento da performance de um modelo após sua implementação;
- Uso de transformadores em _pipelines_ para qualquer tipo de processamento nos dados, garantindo fácil implementação e reprodutibilidade;
- Todos os modelos testados devem obrigatoriamente passar pelos mesmos passos, sendo estes:
    - Divisão dos dados entre treino e validação, estratificado por TARGET;
    - _GridSearchCV_ para hiperparametrização maximizando a _AUC_ (conjunto de treino);
    - Retreino após a escolha dos melhores hiperparâmetros (conjunto de treino);
    - Ajuste do corte de classificação maximizando a função $\frac{90tp - 10fp}{n}$ (conjunto de validação);
    - Retreino após a escolha do melhor corte de classificação e melhores hiperparâmetros (conjunto de treino + validação);
    - Avaliação e comparação no contexto de negócios (conjunto de testes).

## Bibliotecas

In [1]:
# --- Data Exploration and Viz --- #
import pandas as pd

# --- Models --- #
import pickle
from resources.train_evaluate import TrainEvaluate

## Leitura dos dados

In [2]:
test = pd.read_csv("data/test.csv")

## Modelagem

### Random Forest

In [3]:
with open("models/rf.pkl", "rb") as f:
    rf = pickle.load(f)

rf = rf.evaluate(test)

In [4]:
rf.best_model_

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('prep',
                                  Pipeline(steps=[('DropConstantColumns',
                                                   DropConstantColumns(also=['ID'])),
                                                  ('DropDuplicateColumns',
                                                   DropDuplicateColumns()),
                                                  ('NoneZeroCountSaldo',
                                                   AddNonZeroCount(prefix='saldo')),
                                                  ('SumSaldo',
                                                   CustomSum(prefix='saldo')),
                                                  ('NoneZeroCountImp',
                                                   AddNonZeroCount(prefix='imp')),
                                                  ('SumImp',
                                                   CustomSum(prefix='...
                                                   CustomImputer(prefix='var3',
                                                                 to_replace=-999999)),
                                                  ('CustomEncoderVar36',
                                                   CustomEncoder(colname='var36')),
                                                  ('CustomEncoderVar21',
                                                   CustomEncoder(colname='var21'))])),
                                 ('NoneCountVar3', AddNoneCount(prefix='var3')),
                                 ('nan', SimpleImputer(strategy='median'))])),
                ('classifier',
                 RandomForestClassifier(class_weight='balanced', max_depth=8,
                                        max_features=128, n_estimators=500,
                                        random_state=42))])

In [5]:
rf.param_grid

{'classifier__max_depth': [4, 8, 16, 32],
 'classifier__max_features': [8, 16, 32, 64, 96, 128]}

### Histogram Gradient Boosting

In [6]:
with open("models/hgb.pkl", "rb") as f:
    hgb = pickle.load(f)

hgb = hgb.evaluate(test)

In [7]:
hgb.best_model_

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('DropConstantColumns',
                                  DropConstantColumns(also=['ID'])),
                                 ('DropDuplicateColumns',
                                  DropDuplicateColumns()),
                                 ('NoneZeroCountSaldo',
                                  AddNonZeroCount(prefix='saldo')),
                                 ('SumSaldo', CustomSum(prefix='saldo')),
                                 ('NoneZeroCountImp',
                                  AddNonZeroCount(prefix='imp')),
                                 ('SumImp', CustomSum(prefix='imp')),
                                 ('ImputeNanDelta'...
                                  CustomImputer(prefix='var3',
                                                to_replace=-999999)),
                                 ('CustomEncoderVar36',
                                  CustomEncoder(colname='var36')),
                                 ('CustomEncoderVar21',
                                  CustomEncoder(colname='var21'))])),
                ('classifier',
                 HistGradientBoostingClassifier(categorical_features=['var36',
                                                                      'var21'],
                                                class_weight='balanced',
                                                l2_regularization=3,
                                                learning_rate=0.03, max_depth=4,
                                                max_iter=150,
                                                random_state=42))])

In [8]:
hgb.param_grid

{'classifier__learning_rate': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1],
 'classifier__max_iter': [50, 75, 100, 125, 150, 200, 500, 1000],
 'classifier__max_depth': [2, 3, 4, 5, 6, 8],
 'classifier__l2_regularization': [0, 0.1, 0.3, 1, 3]}

## Comparação

Após o treinamento dos dois modelos, temos o _Gradient Boosting_ por histogramas como vencedor. Neste caso, a malha de hiperparâmetros passada para o _Gradient Boosting_ contempla muito mais combinações de hiperparâmetros do que a passada para a _Random Forest_. Inclusive, o melhor número de variáveis aleatórias por split foi o máximo oferecido na malha, indicando que o valor ótimo poderia ser maior.

In [9]:
business_results = pd.DataFrame(
    data = [
        rf.business_metrics,
        hgb.business_metrics
    ],
    index = [
        "Random Forest",
        "Histogram-based Gradient Boosting"
    ]
)

business_results.transpose()

,Random Forest,Histogram-based Gradient Boosting
Profit (Total),14560.000000,15600.000000
Profit (per Customer),0.957643,1.026046
True Positive Profit (Total),32940.000000,29430.000000
True Positive Profit (per Customer),2.166535,1.935675
False Positive Loss (Total),18380.000000,13830.000000
False Positive Loss (per Customer),1.208892,0.909629
False Negative Potential Profit Loss (Total),21240.000000,24750.000000
False Negative Potential Profit Loss (per Customer),1.397001,1.627861
True Negative Loss Prevention (Total),127640.000000,132190.000000
True Negative Loss Prevention (per Customer),8.395159,8.694423


In [10]:
business_results = pd.DataFrame(
    data = [
        rf.classification_metrics,
        hgb.classification_metrics
    ],
    index = [
        "Random Forest",
        "Histogram-based Gradient Boosting"
    ]
)

business_results.transpose()

,Random Forest,Histogram-based Gradient Boosting
Classification Threshold,0.623506,0.741722
ROC AUC,0.844998,0.850288
Precision,0.166062,0.191228
Recall,0.607973,0.543189
F1,0.260870,0.282872
Accuracy,0.863589,0.890950


## Variáveis mais importantes do modelo campeão

In [11]:
feature_importances = hgb \
    .get_feature_importances(test) \
    .reset_index(drop=True)

In [12]:
feature_importances[feature_importances['Importance'] > 0]

,Feature,Importance
0,var15,27334.333333
1,saldo_var30,14844.333333
2,saldo_medio_var5_ult1,1976.666667
3,saldo_var5,1632.000000
4,var38,1531.666667
5,num_var45_hace3,1267.666667
6,saldo_medio_var5_hace3,1075.000000
7,sum_of_num,943.333333
8,saldo_medio_var5_hace2,903.000000
9,num_var22_ult3,643.666667


In [13]:
feature_importances \
    .where(
        lambda ldf:
        ldf["Feature"].apply(
            lambda row:
            (
                row.startswith("none_")
                | row.startswith("sum_")
                | row.startswith("non_")
            )
        )
    ) \
    .dropna()

,Feature,Importance
7,sum_of_num,943.333333
11,non_zero_count_num,409.333333
13,non_zero_count_ind,385.333333
15,sum_of_saldo,303.333333
18,sum_of_imp,132.666667
123,none_count_delta,0.000000
124,non_zero_count_delta,0.000000
296,sum_of_delta,-28.666667
299,non_zero_count_saldo,-35.000000
302,non_zero_count_imp,-51.333333


## Rankeamento da base de testes

In [14]:
test \
    .copy() \
    .assign(predicted=hgb.predict(test.drop("TARGET",axis=1))) \
    .assign(probability=hgb.predict_proba(test.drop("TARGET",axis=1))) \
    .assign(
        profit=(
            lambda ldf:
            ((ldf["TARGET"] * 100) - 10) * ldf["predicted"]
        )
    ) \
    .assign(rank=hgb.rank_customers(test)) \
    [["rank","profit","probability"]] \
    .groupby("rank") \
    .agg({
        "profit": ["mean","sum","count"],
        "probability": ["min"]
    }) \
    .droplevel(0,axis=1) \
    .rename(
        mapper={
            "mean": "avg_profit",
            "sum": "total_profit",
            "count": "total_customers",
            "min": "min_probability"
        },
        axis=1
    )

,avg_profit,total_profit,total_customers,min_probability
rank,,,,
1,9.122807,15600,1710,0.741728
2,0.000000,0,1528,0.556500
3,0.000000,0,2397,0.371012
4,0.000000,0,4046,0.185502
5,0.000000,0,5523,0.022974
